In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Setting Mediapipe functions

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connectionss

In [5]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh])

# Creating Dataset

In [9]:
import os
import mediapipe as mp

labels = ['love','peace','hello','Nothing']
number_img = 100
DATA_PATH = os.path.join('New_Dataset') 
for action in labels: 
    try: 
        os.makedirs(os.path.join(DATA_PATH, action))
    except:
        pass

In [64]:
import time
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for label in labels:
        time.sleep(5)
        print(label)
        for num in range(number_img):
            ret, frame = cap.read()
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
#                 print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            image = cv2.flip(image,1)
            cv2.imshow('OpenCV Feed', image)

            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, label, str(num))
            np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                    
    cap.release()
    cv2.destroyAllWindows()

love
peace
hello
Nothing


In [65]:
cap.release()
cv2.destroyAllWindows()

# Model Creation

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(labels)}

In [11]:
sequences, labeling = [], []
for action in labels:
    for sequence in range(number_img):
        res = np.load(os.path.join(DATA_PATH, action, "{}.npy".format(str(sequence))))
        sequences.append(res)
        labeling.append(label_map[action])

In [12]:
X = np.array(sequences)

In [13]:
X.shape

(400, 63)

In [14]:
y = np.array(labeling)
print(y.shape)

(400,)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [16]:
model_save_path = 'Dataset/keypoint_classifier_new2.h5'

In [17]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((63, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(40, activation='softmax')
])

In [18]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [19]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense (Dense)               (None, 200)               12800     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 40)                4040      
                                                                 
Total params: 36,940
Trainable params: 36,940
Non-trainable params: 0
_________________________________________________________________


# Training the model

In [78]:
hist=model.fit(X_train,y_train,epochs=2000,batch_size=128,validation_data=(X_test, y_test),callbacks=[cp_callback, es_callback])

Epoch 1/2000
1/3 [=========>....................] - ETA: 1s - loss: 3.7175 - accuracy: 0.0000e+00
Epoch 1: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 1s 176ms/step - loss: 3.5943 - accuracy: 0.0526 - val_loss: 3.4325 - val_accuracy: 0.1000
Epoch 2/2000
1/3 [=========>....................] - ETA: 0s - loss: 3.3011 - accuracy: 0.2344
Epoch 2: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 43ms/step - loss: 3.1999 - accuracy: 0.2184 - val_loss: 3.1028 - val_accuracy: 0.3500
Epoch 3/2000
1/3 [=========>....................] - ETA: 0s - loss: 2.9367 - accuracy: 0.2578
Epoch 3: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 38ms/step - loss: 2.8397 - accuracy: 0.2500 - val_loss: 2.7818 - val_accuracy: 0.2500
Epoch 4/2000
1/3 [=========>....................] - ETA: 0s - loss: 2.6031 - accuracy: 0.2734
Epoch 4: saving model to Dataset\keypoint_classifier

Epoch 30/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.8125 - accuracy: 0.7578
Epoch 30: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 48ms/step - loss: 0.8124 - accuracy: 0.7289 - val_loss: 0.6427 - val_accuracy: 1.0000
Epoch 31/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.7777 - accuracy: 0.7188
Epoch 31: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 46ms/step - loss: 0.7567 - accuracy: 0.7289 - val_loss: 0.6260 - val_accuracy: 1.0000
Epoch 32/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.7461 - accuracy: 0.6797
Epoch 32: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 53ms/step - loss: 0.7598 - accuracy: 0.6658 - val_loss: 0.5852 - val_accuracy: 1.0000
Epoch 33/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.6619 - accuracy: 0.8047
Epoch 33: saving model to Dataset\keypoint_classif

Epoch 59/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.4182 - accuracy: 0.8047
Epoch 59: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 50ms/step - loss: 0.3960 - accuracy: 0.8263 - val_loss: 0.1942 - val_accuracy: 1.0000
Epoch 60/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.4699 - accuracy: 0.8047
Epoch 60: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 65ms/step - loss: 0.4299 - accuracy: 0.8289 - val_loss: 0.1690 - val_accuracy: 1.0000
Epoch 61/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.4529 - accuracy: 0.8281
Epoch 61: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 60ms/step - loss: 0.4345 - accuracy: 0.8395 - val_loss: 0.1611 - val_accuracy: 1.0000
Epoch 62/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.3019 - accuracy: 0.8984
Epoch 62: saving model to Dataset\keypoint_classif

Epoch 88/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.3168 - accuracy: 0.8750
Epoch 88: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 58ms/step - loss: 0.3414 - accuracy: 0.8658 - val_loss: 0.0901 - val_accuracy: 1.0000
Epoch 89/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.3674 - accuracy: 0.8594
Epoch 89: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 61ms/step - loss: 0.3249 - accuracy: 0.8605 - val_loss: 0.0797 - val_accuracy: 1.0000
Epoch 90/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.3150 - accuracy: 0.8750
Epoch 90: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 48ms/step - loss: 0.3099 - accuracy: 0.8868 - val_loss: 0.0733 - val_accuracy: 1.0000
Epoch 91/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.3254 - accuracy: 0.8594
Epoch 91: saving model to Dataset\keypoint_classif

Epoch 117/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1709 - accuracy: 0.9609
Epoch 117: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 57ms/step - loss: 0.1888 - accuracy: 0.9421 - val_loss: 0.0399 - val_accuracy: 1.0000
Epoch 118/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.2303 - accuracy: 0.9062
Epoch 118: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 79ms/step - loss: 0.2026 - accuracy: 0.9211 - val_loss: 0.0332 - val_accuracy: 1.0000
Epoch 119/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1821 - accuracy: 0.9453
Epoch 119: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 80ms/step - loss: 0.1874 - accuracy: 0.9316 - val_loss: 0.0559 - val_accuracy: 1.0000
Epoch 120/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.2512 - accuracy: 0.8828
Epoch 120: saving model to Dataset\keypoint

3/3 [==============================] - 0s 58ms/step - loss: 0.1766 - accuracy: 0.9421 - val_loss: 0.0257 - val_accuracy: 1.0000
Epoch 146/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.2382 - accuracy: 0.9062
Epoch 146: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 68ms/step - loss: 0.1916 - accuracy: 0.9342 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 147/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1504 - accuracy: 0.9453
Epoch 147: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 59ms/step - loss: 0.1483 - accuracy: 0.9553 - val_loss: 0.0229 - val_accuracy: 1.0000
Epoch 148/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1035 - accuracy: 0.9531
Epoch 148: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 41ms/step - loss: 0.1468 - accuracy: 0.9395 - val_loss: 0.0217 - val_accuracy: 1.0000
Epoch 149/2

Epoch 203/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.0817 - accuracy: 0.9688
Epoch 203: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 60ms/step - loss: 0.1025 - accuracy: 0.9684 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 204/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1349 - accuracy: 0.9531
Epoch 204: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 48ms/step - loss: 0.1116 - accuracy: 0.9658 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 205/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.0609 - accuracy: 1.0000
Epoch 205: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 42ms/step - loss: 0.1085 - accuracy: 0.9711 - val_loss: 0.0159 - val_accuracy: 1.0000
Epoch 206/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1117 - accuracy: 0.9453
Epoch 206: saving model to Dataset\keypoint

3/3 [==============================] - 0s 46ms/step - loss: 0.0830 - accuracy: 0.9763 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 232/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.0831 - accuracy: 0.9766
Epoch 232: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 51ms/step - loss: 0.0825 - accuracy: 0.9658 - val_loss: 0.0048 - val_accuracy: 1.0000
Epoch 233/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1145 - accuracy: 0.9688
Epoch 233: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 45ms/step - loss: 0.0757 - accuracy: 0.9816 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 234/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.0682 - accuracy: 0.9688
Epoch 234: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 59ms/step - loss: 0.0639 - accuracy: 0.9816 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 235/2

Epoch 289/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.0826 - accuracy: 0.9688
Epoch 289: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 52ms/step - loss: 0.0798 - accuracy: 0.9737 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 290/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1124 - accuracy: 0.9531
Epoch 290: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 47ms/step - loss: 0.0931 - accuracy: 0.9632 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 291/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.0596 - accuracy: 0.9922
Epoch 291: saving model to Dataset\keypoint_classifier_new2.h5
3/3 [==============================] - 0s 44ms/step - loss: 0.0700 - accuracy: 0.9868 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 292/2000
1/3 [=========>....................] - ETA: 0s - loss: 0.1254 - accuracy: 0.9531
Epoch 292: saving model to Dataset\keypoint

# Saving the model

In [79]:
model.save('action2.h5')

# Loading the model

In [21]:
model.load_weights('action2.h5')

# Detecting in real time

In [22]:
import time
threshold = 0.8

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    info_text = ''
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic) 
        draw_styled_landmarks(image, results)
        keypoints = extract_keypoints(results)
        res = model.predict(np.expand_dims(keypoints, axis=0))[0]
#         res = model.predict(keypoints)
        print(res)
        info_text = labels[np.argmax(res)]
        if(info_text == 'Nothing'):
            info_text = '';
        print(info_text)
#         hand_sign_id = keypoint_classifier(keypoints)
#         info_text=unicodedata.name(labels[hand_sign_id])
        image = cv2.flip(image,1)
        cv2.putText(image, info_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
#         cv2.waitKey(1000)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 450ms/step
[8.38670132e-07 2.82983874e-05 3.00702755e-04 9.99666333e-01
 6.14739477e-08 1.38375960e-08 1.39277645e-07 7.56368976e-08
 1.17812206e-07 6.90241464e-08 4.01808755e-08 1.98342704e-07
 1.58519349e-08 1.07222384e-07 1.26182798e-07 2.68417928e-08
 2.45610124e-07 3.09973629e-08 1.39504422e-07 4.58394460e-08
 1.82705847e-07 5.59633797e-08 3.16868984e-08 1.28620499e-07
 9.09193503e-08 1.22715093e-07 8.36807459e-08 1.70122007e-07
 7.28865075e-08 7.65389956e-08 1.77454638e-07 2.57174889e-07
 3.34036741e-07 5.08369027e-08 8.36649505e-08 1.46944984e-07
 5.31993400e-08 1.10066146e-07 1.28206565e-07 7.29777625e-08]

1/1 [==============================] - 0s 64ms/step
[8.38670132e-07 2.82983874e-05 3.00702755e-04 9.99666333e-01
 6.14739477e-08 1.38375960e-08 1.39277645e-07 7.56368976e-08
 1.17812206e-07 6.90241464e-08 4.01808755e-08 1.98342704e-07
 1.58519349e-08 1.07222384e-07 1.26182798e-07 2.68417928e-08
 2.45610124e-07 3.09973629e-08 1.395044

1/1 [==============================] - 0s 32ms/step
[8.38670132e-07 2.82983874e-05 3.00702755e-04 9.99666333e-01
 6.14739477e-08 1.38375960e-08 1.39277645e-07 7.56368976e-08
 1.17812206e-07 6.90241464e-08 4.01808755e-08 1.98342704e-07
 1.58519349e-08 1.07222384e-07 1.26182798e-07 2.68417928e-08
 2.45610124e-07 3.09973629e-08 1.39504422e-07 4.58394460e-08
 1.82705847e-07 5.59633797e-08 3.16868984e-08 1.28620499e-07
 9.09193503e-08 1.22715093e-07 8.36807459e-08 1.70122007e-07
 7.28865075e-08 7.65389956e-08 1.77454638e-07 2.57174889e-07
 3.34036741e-07 5.08369027e-08 8.36649505e-08 1.46944984e-07
 5.31993400e-08 1.10066146e-07 1.28206565e-07 7.29777625e-08]

1/1 [==============================] - 0s 32ms/step
[8.38670132e-07 2.82983874e-05 3.00702755e-04 9.99666333e-01
 6.14739477e-08 1.38375960e-08 1.39277645e-07 7.56368976e-08
 1.17812206e-07 6.90241464e-08 4.01808755e-08 1.98342704e-07
 1.58519349e-08 1.07222384e-07 1.26182798e-07 2.68417928e-08
 2.45610124e-07 3.09973629e-08 1.3950442

1/1 [==============================] - 0s 32ms/step
[7.1421295e-01 2.8319216e-01 2.3404164e-03 1.9388608e-07 2.4961129e-05
 4.7345611e-06 3.3512288e-06 6.9396770e-06 2.0742405e-05 1.7678523e-06
 8.8961424e-06 2.3494217e-06 1.0551728e-06 6.7741439e-06 1.9288143e-06
 2.2095498e-06 1.4291145e-06 1.0278835e-05 3.2730670e-06 2.3608607e-06
 7.9369474e-06 2.9134644e-06 3.7989425e-06 7.8580397e-06 2.8682898e-06
 1.9796230e-06 1.3414588e-06 1.2380210e-05 1.7718314e-05 7.2565899e-06
 6.9399480e-06 1.1718620e-05 1.5370906e-05 7.0076686e-07 6.3939106e-06
 4.6328664e-06 7.3035540e-07 2.1337428e-06 1.4031560e-05 2.2557491e-05]
love
1/1 [==============================] - 0s 24ms/step
[6.4357829e-01 3.5114330e-01 4.9444581e-03 3.7463656e-07 3.0261004e-05
 6.1415667e-06 4.6194359e-06 9.3571380e-06 2.7249880e-05 2.3227628e-06
 1.1643026e-05 3.0736473e-06 1.3880200e-06 8.4800840e-06 2.5888285e-06
 2.9886596e-06 1.9640338e-06 1.2568647e-05 4.4002991e-06 3.0482888e-06
 1.1198804e-05 3.8340690e-06 5.2537321

1/1 [==============================] - 0s 48ms/step
[2.50181556e-02 9.53247309e-01 2.15458274e-02 1.97531335e-06
 9.55968790e-06 2.32347816e-06 3.44731143e-06 6.58134559e-06
 1.77384463e-05 1.23397706e-06 6.27400186e-06 1.72625073e-06
 7.87402598e-07 4.14887154e-06 1.53075825e-06 1.74299839e-06
 1.57008083e-06 5.28347618e-06 2.48148808e-06 1.44665535e-06
 1.03230896e-05 1.99237843e-06 3.01075829e-06 4.64887398e-06
 3.48842286e-06 2.60265870e-06 1.08008396e-06 8.55236522e-06
 1.04441870e-05 4.10681287e-06 6.64898744e-06 1.21374305e-05
 9.80157620e-06 6.30027273e-07 4.69751376e-06 7.45520310e-06
 7.35179185e-07 1.71794716e-06 1.15840139e-05 1.31598872e-05]
peace
1/1 [==============================] - 0s 57ms/step
[2.61188764e-02 9.48443592e-01 2.52364725e-02 2.19766639e-06
 1.01094383e-05 2.49113259e-06 3.67339226e-06 7.00027249e-06
 1.88094273e-05 1.31496245e-06 6.67447921e-06 1.82741780e-06
 8.34342131e-07 4.36859818e-06 1.63300649e-06 1.86231637e-06
 1.67073279e-06 5.52431129e-06 2.65

1/1 [==============================] - 0s 24ms/step
[9.8401272e-01 1.5913514e-02 5.0099763e-05 3.2718950e-09 3.7509799e-06
 4.9292515e-07 2.1575951e-07 5.0823047e-07 1.6730349e-06 1.3758789e-07
 7.9124425e-07 1.8233884e-07 7.9596354e-08 6.1610712e-07 1.4823662e-07
 1.5607144e-07 9.0292083e-08 1.1150015e-06 2.7063342e-07 2.2483505e-07
 4.8490415e-07 2.4086890e-07 3.0045825e-07 7.6673717e-07 1.6395346e-07
 9.5899338e-08 9.7933949e-08 1.2067129e-06 1.8111555e-06 7.3790068e-07
 4.6722744e-07 8.1853841e-07 1.4966514e-06 4.3596156e-08 5.1953754e-07
 2.2249003e-07 4.6046456e-08 1.5909019e-07 1.1567067e-06 2.3009829e-06]
love
1/1 [==============================] - 0s 24ms/step
[9.8323071e-01 1.6691750e-02 5.2894618e-05 3.4958803e-09 3.8986482e-06
 5.1474404e-07 2.2682792e-07 5.3379932e-07 1.7520892e-06 1.4418184e-07
 8.2790706e-07 1.9116588e-07 8.3680980e-08 6.4486204e-07 1.5567296e-07
 1.6375209e-07 9.5132386e-08 1.1646114e-06 2.8371886e-07 2.3546704e-07
 5.1028991e-07 2.5265976e-07 3.1500201

1/1 [==============================] - 0s 24ms/step
[3.8994946e-02 2.6419589e-01 6.9635892e-01 1.3104174e-05 1.8082606e-05
 6.7795158e-06 8.0856689e-06 1.3267927e-05 3.5546320e-05 2.7589185e-06
 1.4149574e-05 3.2753783e-06 1.5597824e-06 7.6280580e-06 3.7367915e-06
 4.3303080e-06 3.5478431e-06 7.5694970e-06 6.3280327e-06 3.2701973e-06
 2.6663134e-05 4.9368523e-06 1.0716288e-05 8.6333257e-06 7.5220528e-06
 7.8112244e-06 3.4737182e-06 2.2449462e-05 2.7430397e-05 1.0818376e-05
 1.9844370e-05 2.7126727e-05 2.7496922e-05 1.5605190e-06 5.8701371e-06
 1.1690294e-05 2.0404445e-06 3.3273006e-06 3.9096667e-05 2.8856350e-05]
hello
1/1 [==============================] - 0s 24ms/step
[9.90650415e-01 9.31517873e-03 2.25771128e-05 6.83457346e-10
 1.97530949e-06 2.55648757e-07 9.83950486e-08 2.36351923e-07
 8.73484055e-07 6.59573374e-08 3.92677777e-07 8.29352089e-08
 3.64747379e-08 3.23605803e-07 6.81666634e-08 7.21360820e-08
 3.98616464e-08 5.62719947e-07 1.20787462e-07 1.07771264e-07
 2.21128914e-07 

In [88]:
cap.release()
cv2.destroyAllWindows()

In [62]:
model.predict(np.expand_dims(keypoints, axis=0))[0]

1/1 [==============================] - 0s 52ms/step


array([0.012135  , 0.05804422, 0.2677248 , 0.01928562, 0.02470947,
       0.01637675, 0.01783084, 0.01673577, 0.0189481 , 0.01589733,
       0.02432427, 0.01756183, 0.01581371, 0.01604193, 0.02150793,
       0.01525534, 0.01840544, 0.01170744, 0.01407549, 0.01701135,
       0.01819286, 0.0140328 , 0.0198866 , 0.01922262, 0.02067938,
       0.01538338, 0.02146728, 0.01504121, 0.01561783, 0.01375651,
       0.01775732, 0.01748059, 0.01771153, 0.01953802, 0.01623498,
       0.02627269, 0.0160705 , 0.01698272, 0.01860377, 0.02067475],
      dtype=float32)